In [1]:
!pip install pytorch-lightning --q

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pytorch_lightning as pl

In [3]:
transform = transforms.ToTensor()
train_ds = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_ds = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [4]:
train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
valid_dl = DataLoader(test_ds, batch_size=128, shuffle=False)

In [5]:
class MNIST_Classifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        logists = self(images)
        loss = F.cross_entropy(logists, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logists = self(images)
        loss = F.cross_entropy(logists, labels)
        acc = (logists.argmax(dim=1) == labels).float().mean()
        self.log('valid_loss', loss)
        self.log('valid_acc', acc)
       # return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


In [ ]:
model = MNIST_Classifier()
trainer = pl.Trainer(max_epochs=10, log_every_n_steps=50)
trainer.fit(model, train_dl, valid_dl)

In [ ]:
# Access the validation and training losses
train_losses = trainer.callback_metrics['train_loss'].cpu().numpy()
valid_losses = trainer.callback_metrics['valid_loss'].cpu().numpy()
valid_acc = trainer.callback_metrics['valid_acc'].cpu().numpy()
print(f"Final Train Loss: {train_losses}")
print(f"Final Validation Loss: {valid_losses}")
print(f"Final Validation Accuracy: {valid_acc}")

# Getting predictions on the test set
model.eval() # Set the model to evaluation mode
predictions = []
true_labels = []

with torch.no_grad(): # Disable gradient calculation
    for images, labels in valid_dl: # Use the validation dataloader for prediction
        logists = model(images)
        preds = logists.argmax(dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print("\nSample Predictions:")
print(predictions[:10])
print("Sample True Labels:")
print(true_labels[:10])

In [9]:
#!jupyter nbconvert --clear-output --inplace W3DAY16_PyTorchLightning.ipynb

[NbConvertApp] WARNING | pattern 'W3DAY16_PyTorchLightning.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
